# ChildMind File Organization / Mapping

Words, words, words.

### Event code descriptions:

__Contrats Change__
- 3 :  5  = start trial
- 4 :  8  = Target ON left
- 5 :  9  = Target ON right
- 1 : 12  = button press left
- 2 : 13  = button press right

__Naturalistic Viewing Paradigm__
- 33 :  81  = start of video 1
- 1 :  101  = stop of video 1
- 34 :  82  = start of video 2
- 2 :  102  = stop of video 2
- 35 :  83  = start of video 3
- 3 :  103  = stop of video 3


__Resting Paradigm__
- 1 :  20 = eyes open start
- 2 :  30 = eyes closed start


__Sequence Learning Paradigm__

- *DOT ON:*
- 6 : 11
- 7 : 12
- 8 : 13
- 9 : 14
- 10 : 15
- 11 : 16
- 12 : 17
- 13 : 18

- *DOT OFF:*
- 15 : 21
- 16 : 22
- 17 : 23 
- 18 : 24
- 19 : 25
- 20 : 26
- 21 : 27
- 22 : 28


__Surround Suppression Paradigm__
- 1 :  4 = stimulus ON
- 2 :  8 = stimulus OFF
(There are also two condotions in this experiment which says there's a background or there is no background)


__Symbol Serach Paradigm__
- 2 :  20 = start of new page
- 1 :  14 = response for trial



In [ ]:
import os
import csv

from collections import Counter

from mne import set_log_level
from mne.io import read_raw_egi

In [ ]:
# Set MNE logging to only print critical issues
set_log_level('CRITICAL')

In [ ]:
# Create a mapping of event codes to file type
events = {
    'contrast'  : [{'12' : 1, '13': 2, '5': 3, '8': 4, '9': 5}],
    
    'nat_view'  : [{'81': 33, '101':1, '82':34, '102':2, '83':35, '103':3}],
    
    'resting'   : [{'20': 1, '30' : 2}, {'20': 1, '30' : 2, '90' : 3}],
    
    'seq_learn' : [{'11' :  1, '12':  2, '13' :  3, '14' :  4, '15' :  5, '16' :  6,
                    '17' :  7, '18':  8, '21' :  9, '22' : 10, '23' : 11, '24' : 12,
                    '25' : 13, '26': 14, '27' : 15, '28' : 16}],
    
    'surr_supp' : [{'4' : 1,  '8' : 2}],
    
    'symb_srch' : [{'14': 1, '20' : 2}],
    
    'movie'     : [{'101' :  1, '102' :  2, '103' :  3, '104' :  4, '106':  5, '11' :  6, '12':  7,
                     '13' :  8,  '14' :  9,  '15' : 10,  '16' : 11,  '17': 12, '18' : 13, '20': 14,
                     '21' : 15,  '22' : 16,  '23' : 17,  '24' : 18,  '25': 19, '26' : 20, '27': 21,
                     '28' : 22,  '30' : 23,  '31' : 24,  '32' : 25,  '33': 26, '34' : 27, '35': 28,
                      '4' : 29,   '5' : 30,  '50' : 31,   '8' : 32,  '81': 33, '82' : 34, '83': 35,
                     '84' : 36,  '86' : 37,   '9' : 38,  '90' : 39,  '91': 40, '92' : 41, '93': 42,
                     '94' : 43,  '95' : 44,  '96' : 45,  '97' : 46}]
}

# Load Data

In [ ]:
# Set data path
#dat_path = '/Users/trahimi1/Downloads/MIND_2018/Hackathon/Subjects/'
dat_path = '/Users/tom/Desktop/HyperEEG_Project/Data/raw/'

In [ ]:
# Get list of all available subjects
subj_nums = [file for file in os.listdir(dat_path) if file[0] is not '.']

In [ ]:
def cln_dict(evd):
    return {ke.strip(): va for ke, va in evd.items()}

In [ ]:
file_labels = []
for sub_num in subj_nums:

    subj_path = os.path.join(dat_path, sub_num, 'EEG', 'raw', 'raw_format')
    subj_files = [file for file in os.listdir(subj_path) if ('A0' in file and file.endswith('.raw'))]

    print('Running Subject #', subj_file.split('.')[0])

    for ind, subj_file in enumerate(subj_files):

        raw = mne.io.read_raw_egi(os.path.join(subj_path, subj_file), preload=False)

        cur_task = None

        for key, vals in events.items():
            
            for val in vals:
                if cln_dict(raw.event_id) == val:
                    #print('\tFound event:', key)
                    cur_task = key
                    break

        if not cur_task:
            print(raw.event_id)
            print('\tNo match found for file.')

        file_labels.append((subj_file[0:9], subj_file, cur_task))

In [ ]:
# Save out a file of all the mappings
with open('file_mappings.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    for f_info in file_labels:
        writer.writerow(list(f_info))